In [31]:
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from skimage.segmentation import mark_boundaries 






def get_image_value(path, dim): 
    '''This function will read an image and convert to a specified version and resize 
    depending on which algorithm is being used. '''
    img = image.load_img(path, target_size = dim)
    img = image.img_to_array(img)
    return img/255

def get_img_array(img_paths, dim): 
    '''This fucntion takes a list of image paths and returns the np array corresponding to each image.  
    It also takes the dim and whether edge is specified in order to pass it to another function to apply these parameters.  
    This function uses get_image_value to perform these operations'''
    final_array = []
    from tqdm import tqdm
    for path in tqdm(img_paths):
        img = get_image_value(path, dim)
        final_array.append(img)
    final_array = np.array(final_array)  
    return final_array

def get_tts():
    '''This function will create a train test split'''  
   
    DIM =  (150,150) 
    np.random.seed(10)        
    pistol_paths = [path + 'Pistol//' + i for i in os.listdir(path + 'Pistol')] 
    pistol_labels = [1 for i in range(len(pistol_paths))]
    rifle_paths = [path + 'Rifle//' + i  for i in os.listdir(path + 'Rifle')] 
    rifle_labels = [2 for i in range(len(rifle_paths))]    
    neg_paths = [path + 'NoWeapon//' + i for i in os.listdir(path + 'NoWeapon')]
    np.random.shuffle(neg_paths)
    neg_paths = neg_paths[:len(pistol_paths)- 500]
    neg_labels = [0 for i in range(len(neg_paths))]

    np.random.shuffle(pistol_paths)
    pistol_paths = pistol_paths[:len(rifle_paths)+150]
    neg_paths = neg_paths[:len(rifle_paths)+150]

    pistol_labels = [1 for i in range(len(pistol_paths))]
    rifle_labels = [2 for i in range(len(rifle_paths))]
    neg_labels = [0 for i in range(len(neg_paths))]
    paths = pistol_paths + rifle_paths + neg_paths
#     paths = pistol_paths 
    
#     labels =  pistol_labels 
    labels = pistol_labels + rifle_labels + neg_labels
    
    x_train, x_test, y_train, y_test = train_test_split(paths, labels, stratify = labels, train_size = .90, random_state = 10)

    new_x_train = get_img_array(x_train, DIM)
    new_x_test = get_img_array(x_test, DIM)
    
    print('Train Value Counts')
    print(pd.Series(y_train).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test Value Counts')
    print(pd.Series(y_test).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Train Shape')
    print(new_x_train.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Test Shape')
    print(new_x_test.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)
    tts = (new_x_train, new_x_test, y_train, y_test)
    return tts

path = 'D:/OneDrive - CGIAR/PROYECTOS/Croppie/Tutoriales/Separated/FinalImages/'

x_train, x_test, y_train, y_test = get_tts()

#uncomment the code below to see what the images look like
#cv2.imshow('test', x_train[25])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

100%|███████████████████████████████████████████████████████████████████████████████| 441/441 [00:01<00:00, 410.76it/s]


Train Value Counts
0    1368
1    1368
2    1233
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test Value Counts
0    152
1    152
2    137
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Train Shape
(3969, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Test Shape
(441, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [33]:
#uncomment the code below to see what the images look like
cv2.imshow('test', x_train[44])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [56]:
import numpy as np 
#from keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,  BatchNormalization, AveragePooling2D, Dense, Dropout, Flatten 

from keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle

#from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense




def get_conv_model(dim = (150,150, 3)):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)    
    model = Sequential() 
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3))) 
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))  
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 

In [57]:
#prevents overfitting and saves models every time the validation loss improves
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 2, mode = 'min')
epochs = 1000
batch_size = 32
model = get_conv_model()
model_history = model.fit(x_train, y_train, batch_size = batch_size,
             epochs = epochs, 
      callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


Epoch 1/1000
124/125 [============================>.] - ETA: 0s - loss: 11.3341 - accuracy: 0.5348
Epoch 00001: val_loss improved from inf to 10.25967, saving model to ModelWeights.h5
125/125 [==============================] - 88s 705ms/step - loss: 11.3339 - accuracy: 0.5346 - val_loss: 10.2597 - val_accuracy: 0.6712
Epoch 2/1000
124/125 [============================>.] - ETA: 0s - loss: 9.4761 - accuracy: 0.6507
Epoch 00002: val_loss improved from 10.25967 to 8.68995, saving model to ModelWeights.h5
125/125 [==============================] - 90s 721ms/step - loss: 9.4757 - accuracy: 0.6508 - val_loss: 8.6899 - val_accuracy: 0.6599
Epoch 3/1000
124/125 [============================>.] - ETA: 0s - loss: 8.0262 - accuracy: 0.6928
Epoch 00003: val_loss improved from 8.68995 to 7.51532, saving model to ModelWeights.h5
125/125 [==============================] - 89s 715ms/step - loss: 8.0261 - accuracy: 0.6926 - val_loss: 7.5153 - val_accuracy: 0.6213
Epoch 4/1000
125/125 [=================

Epoch 27/1000
124/125 [============================>.] - ETA: 0s - loss: 2.0491 - accuracy: 0.8775
Epoch 00027: val_loss improved from 2.31853 to 2.28055, saving model to ModelWeights.h5
125/125 [==============================] - 96s 767ms/step - loss: 2.0492 - accuracy: 0.8773 - val_loss: 2.2805 - val_accuracy: 0.7574
Epoch 28/1000
125/125 [==============================] - ETA: 0s - loss: 2.0093 - accuracy: 0.8728
Epoch 00028: val_loss improved from 2.28055 to 2.19723, saving model to ModelWeights.h5
125/125 [==============================] - 102s 815ms/step - loss: 2.0093 - accuracy: 0.8728 - val_loss: 2.1972 - val_accuracy: 0.7823
Epoch 29/1000
125/125 [==============================] - ETA: 0s - loss: 1.9186 - accuracy: 0.8919
Epoch 00029: val_loss improved from 2.19723 to 2.17186, saving model to ModelWeights.h5
125/125 [==============================] - 104s 832ms/step - loss: 1.9186 - accuracy: 0.8919 - val_loss: 2.1719 - val_accuracy: 0.7596
Epoch 30/1000
125/125 [============

KeyboardInterrupt: 

In [29]:
neg_paths = ['{i}' for i in range(6)]


In [30]:
neg_paths

['{i}', '{i}', '{i}', '{i}', '{i}', '{i}']